In [17]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd

In [18]:
# Step 2: Define function to load from a GeoTIFF file
def load_tif(filepath):
    """Load a single-band TIFF file as a numpy array."""
    with rasterio.open(filepath) as src:
        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
        
        return src.read(1), src.transform, src.crs, extent

In [19]:
def read_kml(ref_tiff_path):
    # Load the KML file
    kml_path = 'dhaka_cIty_gis_project.kml'
    kml_gdf = gpd.read_file(kml_path, driver="KML", allow_unsupported_drivers=True)

    with rasterio.open(ref_tiff_path) as src:
        ref_crs = src.crs

    # Reproject the KML to match the TIFF CRS
    if kml_gdf.crs != ref_crs:
        kml_gdf = kml_gdf.to_crs(ref_crs)

    return kml_gdf

In [20]:
def save_tif(output_path, data, transform, crs):
    """Save a numpy array as a GeoTIFF file."""
    metadata = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'nodata': None,
        'width': data.shape[1],
        'height': data.shape[0],
        'count': 1,
        'crs': crs,
        'transform': transform
    }
    with rasterio.open(output_path, 'w', **metadata) as dst:
        dst.write(data.astype('float32'), 1)
    print(f"Difference GeoTIFF saved to: {output_path}")

In [21]:
def compare_ibi(ibi_2013_path, ibi_2023_path, previous_year, last_year, output_path):
    """
    Compare two IBI GeoTIFF files and visualize the difference.
    
    Parameters:
    - ibi_2013_path: Path to the IBI GeoTIFF for 2013.
    - ibi_2023_path: Path to the IBI GeoTIFF for 2023.
    - output_path: Path to save the difference GeoTIFF (optional).
    """
    # Load the IBI data for both years
    ibi_2013, transform, crs, extent = load_tif(ibi_2013_path)
    ibi_2023, _, _, extent = load_tif(ibi_2023_path)

    kml_gdf = read_kml(ibi_2013_path)

    # Calculate the difference
    ibi_difference = ibi_2023 - ibi_2013

    # Visualize the difference
    plt.figure(figsize=(8, 6))
    plt.imshow(ibi_difference, cmap="RdYlGn", vmin=-1, vmax=1, extent=extent)
    plt.colorbar(label="Difference in IBI (2023 - 2013)")
    plt.title("Difference in Index-based Built-up Index (IBI)")
    plt.xlabel("Pixel Column")
    plt.ylabel("Pixel Row")

    # Overlay the KML polygons
    kml_gdf.boundary.plot(ax=plt.gca(), color='black', linewidth=6)

    output_path = output_path + "IBI_difference_" + last_year + "_" + previous_year
    output_tiff = output_path + ".tif"
    output_png = output_path + ".png"

    # Save the difference GeoTIFF if an output path is provided

    save_tif(output_tiff, ibi_difference, transform, crs)
    plt.savefig(output_png, dpi=300, bbox_inches='tight')

    plt.show()

Difference GeoTIFF saved to: ./Landsat8/output/IBI_difference_2023_2013.tif


In [22]:
# Paths to the IBI GeoTIFF files
ibi_2013_path = "./Landsat8/output/2013/IBI_2013.tif"
ibi_2023_path = "./Landsat8/output/2023/IBI_2023.tif"

# Output path for the difference GeoTIFF
output_path = "./Landsat8/output/"

# Compare IBI and save the difference
compare_ibi(ibi_2013_path, ibi_2023_path,"2013", "2023", output_path)

In [23]:
def calculate_area_change(data, pixel_area):
    """
    Calculate total positive and negative change areas in the raster.

    Parameters:
    - data: Difference array between two IBI rasters.
    - pixel_area: Area of one pixel (e.g., 30x30 m = 900 m²).

    Returns:
    - positive_area: Total area with positive change (urbanization).
    - negative_area: Total area with negative change (deurbanization).
    """
    positive_area = np.sum(data > 0) * pixel_area
    negative_area = np.sum(data < -.5) * pixel_area
    return positive_area, negative_area

def summarize_urban_sprawl(ibi_2013_path, ibi_2023_path, pixel_area, output_difference_path=None):
    """
    Compare two IBI GeoTIFF files, calculate urban sprawl metrics, and save the difference map.

    Parameters:
    - ibi_2013_path: Path to the IBI GeoTIFF for 2013.
    - ibi_2023_path: Path to the IBI GeoTIFF for 2023.
    - pixel_area: Area of one pixel in square meters.
    - output_difference_path: Path to save the difference GeoTIFF (optional).

    Returns:
    - Summary of urban sprawl statistics.
    """
    # Load the IBI data for both years
    ibi_2013, transform, crs, extent = load_tif(ibi_2013_path)
    ibi_2023, _, _, _ = load_tif(ibi_2023_path)

    kml_gdf = read_kml(ibi_2013_path)

    # Calculate the difference
    ibi_difference = ibi_2023 - ibi_2013

    # Calculate positive and negative areas (urbanization and deurbanization)
    positive_area, negative_area = calculate_area_change(ibi_difference, pixel_area)
    total_area = ibi_difference.size * pixel_area
    stable_area = total_area - (positive_area + negative_area)

    # Visualize the difference
    plt.figure(figsize=(8, 6))
    plt.imshow(ibi_difference, cmap="seismic", vmin=-1, vmax=1, extent=extent)
    plt.colorbar(label="Difference in IBI (2023 - 2013)")
    plt.title("Difference in Index-based Built-up Index (IBI)")
    plt.xlabel("Pixel Column")
    plt.ylabel("Pixel Row")
 
    # Overlay the KML polygons
    kml_gdf.boundary.plot(ax=plt.gca(), color='black', linewidth=2)

    plt.show()

    # Print the summary statistics
    print("\nUrban Sprawl Detection Summary (2013 - 2023):")
    print(f"Total Study Area: {total_area / 1e6:.2f} sq. km")
    print(f"Urbanized Area (Positive IBI Change): {positive_area / 1e6:.2f} sq. km ({(positive_area / total_area) * 100:.2f}%)")
    print(f"Deurbanized Area (Negative IBI Change): {negative_area / 1e6:.2f} sq. km ({(negative_area / total_area) * 100:.2f}%)")
    print(f"Stable Area: {stable_area / 1e6:.2f} sq. km ({(stable_area / total_area) * 100:.2f}%)")

Urban Sprawl Detection Summary (2013 - 2023):
Total Study Area: 691.51 sq. km
Urbanized Area (Positive IBI Change): 435.82 sq. km (63.02%)
Deurbanized Area (Negative IBI Change): 107.71 sq. km (15.58%)
Stable Area: 147.99 sq. km (21.40%)


In [24]:
# Parameters
# Landsat resolution (30m x 30m)
pixel_size = 30  

 # Area of one pixel in square meters
pixel_area = pixel_size ** 2 

# Paths to the IBI GeoTIFF files
ibi_2013_path = "./Landsat8/output/2013/IBI_2013.tif"
ibi_2023_path = "./Landsat8/output/2023/IBI_2023.tif"

# Output path for the difference GeoTIFF
output_difference_path = "./Landsat8/output/IBI_difference_2023_2013.tif"

# Run the analysis
summarize_urban_sprawl(ibi_2013_path, ibi_2023_path, pixel_area, output_difference_path)